# Customisable Prediction Accuracy Plots

In this notebook, any specified model which adheres to the sklearn API can be fitted across customisable training windows, with predictions made and plotted for the specified test window. Models are persisted to disk for quickly repeatable runs.

In [19]:
import numpy as np
import pandas as pd
from tscv import GapRollForward
from tqdm.notebook import tqdm
from sklearn.ensemble import HistGradientBoostingRegressor

region = 'sa'

# extract holidays from file
holiday_df = pd.read_csv('../data/holidays2017_2024.csv', dtype='str')
holiday_df['Date'] = holiday_df['Date'].astype('datetime64[ns]').dt.date
holidays = holiday_df.loc[holiday_df['Jurisdiction'] == region, ['Date', 'Holiday Name']]
holidict = {name: i+1 for i, name in enumerate(holidays['Holiday Name'].unique())}

# import and preprocess load and weather data
df = pd.read_csv(f'../data/{region}/merged.csv')
df['datetime'] = df['datetime'].astype('datetime64')
dt = df['datetime'].dt
df['year'] = dt.year
df['month'] = dt.month
df['day'] = dt.day
df['hour'] = dt.hour
df['minute'] = dt.minute
df['dow'] = dt.day_of_week
df['week'] = dt.isocalendar().week

# compute universal X and y column indices
X_EXCLUDE = ['datetime', 'net_load', 'total_load']
X_cols = np.setdiff1d(df.columns.values, X_EXCLUDE)
y_ind = df.columns.get_loc('net_load')

# add holiday encodings
df['holiday_bin'] = dt.date.isin(holidays['Date']).astype('int')
merged = pd.merge(dt.date, holidays, left_on='datetime', right_on='Date', how='left')
df['holiday_cat'] = merged['Holiday Name'].replace(holidict).fillna(0).astype('int')
df['working_day'] = np.logical_not(np.logical_or(df['holiday_bin'], df['dow'] >= 5))

# for convenience below
obs = np.arange(1000) * 24

# create train/test window strategies
tscv_sliding = GapRollForward(
    min_train_size = obs[365], 
    max_train_size = obs[365],
    min_test_size = obs[7], 
    max_test_size = obs[7],
    roll_size = obs[30])

tscv_expanding = GapRollForward(
    min_train_size = obs[365], 
    max_train_size = np.inf,
    min_test_size = obs[7], 
    max_test_size = obs[7],
    roll_size = obs[30])

print(sum(1 for i in tscv_sliding.split(df)), f' sliding windows to be trained')
print(sum(1 for i in tscv_expanding.split(df)), f' expanding windows to be trained')

49  sliding windows to be trained
49  expanding windows to be trained


In [2]:
tscv = {'sliding': tscv_sliding, 'expanding': tscv_expanding}

prdfs = []
# execute train/test window strategies
for strategy, cv in tscv.items():
    for i, (train_ind, test_ind) in tqdm(enumerate(cv.split(df))):
        for include in [[], ['holiday_bin'], ['holiday_cat'], ['working_day']]:
            X_inds = sorted(df.columns.get_indexer_for(X_cols.tolist() + include))

            X_train, X_test = df.iloc[train_ind, X_inds], df.iloc[test_ind, X_inds]
            y_train, y_test = df.iloc[train_ind, y_ind], df.iloc[test_ind, y_ind]

            # train model
            begin, end = df.iloc[[train_ind[0], train_ind[-1]], 0].dt.date
            model = HistGradientBoostingRegressor()
            model.fit(X_train, y_train)

            # predict
            prd = model.predict(X_test)
            prdf = pd.DataFrame({'datetime': df.iloc[test_ind, 0],
                                'model': i,
                                'holiday': include[0] if include else 'no_holiday',
                                'window': strategy,
                                'train_end': end,
                                'predicted': prd,
                                'net_load': y_test})
            prdfs.append(prdf)

# concatenate predictions and compute discrete error metrics
predictions = pd.concat(prdfs)
predictions['Residual'] = predictions['predicted'] - predictions['net_load']
predictions['Absolute Error'] = predictions['Residual'].abs()
predictions['Percent Error'] = predictions['Residual'] / predictions['net_load']
predictions['Absolute Percent Error'] = predictions['Percent Error'].abs()
predictions['Squared Error'] = predictions['Residual'] ** 2

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [3]:
metrics = predictions.groupby(['window', 'holiday']).mean(numeric_only = True).reset_index()
metrics['RMSE'] = np.sqrt(metrics['Squared Error'])
metrics['MAE'] = metrics['Absolute Error']
metrics['MAPE'] = metrics['Absolute Percent Error']

metrics[['window', 'holiday', 'MAE', 'MAPE', 'RMSE']]

,window,holiday,MAE,MAPE,RMSE
0,expanding,holiday_bin,75.526819,0.063135,109.475348
1,expanding,holiday_cat,75.762274,0.063381,109.743669
2,expanding,no_holiday,77.640723,0.065044,112.756050
3,expanding,working_day,75.467718,0.063150,109.181335
4,sliding,holiday_bin,78.641494,0.064572,116.786208
5,sliding,holiday_cat,78.795340,0.064766,117.256788
6,sliding,no_holiday,80.567411,0.066334,119.520325
7,sliding,working_day,77.992834,0.063759,116.410798


In [18]:
st_inds = (predictions['datetime'].dt.date - predictions['train_end']) <= pd.Timedelta('2d')
metrics = predictions.loc[st_inds,:].groupby(['window', 'holiday']).mean(numeric_only = True).reset_index()
metrics['RMSE'] = np.sqrt(metrics['Squared Error'])
metrics['MAE'] = metrics['Absolute Error']
metrics['MAPE'] = metrics['Absolute Percent Error']

metrics[['window', 'holiday', 'MAE', 'MAPE', 'RMSE']]

,window,holiday,MAE,MAPE,RMSE
0,expanding,holiday_bin,80.721263,0.067759,117.147029
1,expanding,holiday_cat,80.626055,0.067869,116.984494
2,expanding,no_holiday,82.211816,0.069216,119.061134
3,expanding,working_day,80.959606,0.068137,116.647670
4,sliding,holiday_bin,85.495200,0.070550,127.219884
5,sliding,holiday_cat,85.294081,0.070638,127.161092
6,sliding,no_holiday,86.625505,0.071493,128.790917
7,sliding,working_day,84.815281,0.069776,127.134633
